# Baseline model

This notebook demonstrates the baseline model for estimating the respiratory rate from a video of a person's chest. The model uses the Viola Jones algorithm to detect the face in the first frame of the video. It then calculates the region of interest (ROI) based on the face position and size. The model calculates the average pixel intensity in the chest region and applies a fast Fourier transform (FFT) to the signal. The respiratory rate is estimated by identifying the frequency that exhibits the highest amplitude within the FFT spectrum.

In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

subject = 'Proband16'
scenario = '101_natural_lighting'

frames, video_params = dataset.get_video_gray(subject, scenario)

## Detect the face in the first frame and calculate the region of interest

In [ ]:
import matplotlib.pyplot as plt
import respiration.roi as roi

faces = roi.detect_faces(frames[0])
if len(faces) == 0:
    raise ValueError('No face detected in the first frame')
elif len(faces) > 1:
    raise ValueError('Multiple faces detected in the first frame')

# First face position and size
face1 = faces[0]

# Calculate the region of interest (ROI) based on the face
roi = roi.roi_from_face(faces[0])

In [ ]:
import cv2

# Detect the face in the first frame with Viola Jones algorithm
frame1 = frames[0].copy()

# Draw a rectangle around the face
cv2.rectangle(frame1, (face1[0], face1[1]), (face1[0] + face1[2], face1[1] + face1[3]), (255, 0, 0), 2)

# Draw a rectangle around the chest region
cv2.rectangle(frame1, (roi[0], roi[1]), (roi[0] + roi[2], roi[1] + roi[3]), (255, 0, 0), 2)

plt.figure(figsize=(5, 5))
plt.imshow(frame1, cmap='gray')
plt.title('Face and chest detection')
plt.show()

## Calculate the average pixel intensity in the chest region

In [ ]:
import respiration.extractor.pixel_intensity as pixel_intensity

# Calculate the average pixel intensity in the chest region
average_pixels = pixel_intensity.average_pixel_intensity(frames, roi)

In [ ]:
import plotly.express as px

# Plot the average pixel intensity in the chest region
fig = px.line(x=range(len(average_pixels)), y=average_pixels,
              title='Average pixel intensity in the chest region')
fig.show()

## Calculate the respiratory rate

In [ ]:
from respiration.analysis import FrequencyExtractor
import respiration.analysis.fft as fft

lowpass = 0.08
highpass = 0.6

# Calculate the fast fourier transform of the average pixel intensity
frequencies, pixels_fft = fft.fft_spectrum(average_pixels, video_params.fps, lowpass, highpass)

In [ ]:
import numpy as np

# Plot the filtered frequency spectrum
fig = px.line(x=frequencies, y=np.abs(pixels_fft), title='Filtered Frequency spectrum')
fig.show()

In [ ]:
# Find the frequency with the highest amplitude
peak_freq = frequencies[np.argmax(np.abs(pixels_fft))]
peak_freq, peak_freq * 60